In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation
import math, random
import os
from torch.utils.data import DataLoader

In [76]:
class Series_dataset(torch.utils.data.Dataset):
    
    def __init__(self, dataset_name):
        
        np_dict = np.load(dataset_name,allow_pickle=True).item()
        sort = np.argsort(np_dict["length"])
        self.items = np_dict["series"].astype(np.float32)[sort]
        self.targets = np_dict["churn"].astype(int)[sort]
        self.length = np_dict["length"][sort]
        self.input_size = np_dict["series"][0].shape[1]
        
    def __len__(self):        
        return self.targets.shape[0]
    
    def __getitem__(self,index):
        
        return self.items[index],self.targets[index],self.length[index]

In [71]:
train = np.load("../data/series_train.npy",allow_pickle=True).item()

In [72]:
length = train["length"]

In [73]:
np.sort(train["length"])

array([ 1,  1,  1, ..., 30, 30, 30])

In [75]:
length[np.argsort(train["length"])]

array([ 1,  1,  1, ..., 30, 30, 30])

In [ ]:
items = train["series"].astype(np.float32)

In [43]:
class RNN(nn.Module):
    def __init__(self,input_size, h_size,layers=1, dropout=0.5,output_dim=2):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
        input_size=input_size,
        hidden_size=h_size, 
        num_layers=layers, 
        batch_first=True,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(h_size, output_dim)
        
    def forward(self, x, length):
        
        out, h_state = self.rnn(x)#, h_state)
        out = self._gather_last_output(out,length)
        
        z = self.dropout(out)
        y_pred = self.fc(z)
        
        return y_pred, h_state
    
    def _gather_last_output(self, output, seq_length):
        seq_length = seq_length.long().detach().cpu().numpy() - 1
        out = []
        for batch_index, column_index in enumerate(seq_length):
            out.append(output[batch_index, column_index])
        return torch.stack(out)    

In [109]:
input_size= 5
h_size = 32
layers=1
dropout=0.5
output_dim = 2
l2_weight = 1e-5
#gpu = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

max_epochs=20
learning_rate = 0.001
batch_size = 256

In [77]:
train_dataset = Series_dataset("../data/series_train.npy")
train_data_loader = DataLoader(train_dataset,batch_size=batch_size,num_workers=8)

In [78]:
val_dataset = Series_dataset("../data/series_val.npy")
val_data_loader = DataLoader(val_dataset,batch_size=batch_size,num_workers=8)

In [111]:
model = RNN(input_size, h_size,layers, dropout,output_dim)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [107]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    #rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = np.array(preds.max(1)[1].data.tolist())
    #print(preds.shape)
    #print(rounded_preds.shape,y.shape)
    correct = (rounded_preds == np.array(y.cpu()))#.float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [82]:
for batch in train_data_loader:
    print(batch[0].shape)
    break

torch.Size([256, 30, 5])


In [92]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        x = batch[0].to(device)
        y = batch[1].to(device)
        length = batch[2].to(device)
        optimizer.zero_grad()
        predictions,h_state = model(x,length)#.squeeze(1)
        loss = criterion(predictions, y)
        acc = binary_accuracy(predictions, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator),epoch_acc / len(iterator)

In [93]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            x = batch[0].to(device)
            y = batch[1].to(device)
            length = batch[2].to(device)
        
            predictions,h_state = model(x,length)#.squeeze(1)
            loss = criterion(predictions, y)
            acc = binary_accuracy(predictions, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator),epoch_acc / len(iterator)

In [85]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [112]:
best_valid_loss = float('inf')

for epoch in range(max_epochs):

    start_time = time.time()
    
    train_loss,train_acc = train(model, train_data_loader, optimizer, criterion)
    valid_loss ,valid_acc = evaluate(model, val_data_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'RNN-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f})| Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f}) |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.679)| Train Acc: 58.03%
	 Val. Loss: 0.648) |  Val. Acc: 63.33%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.632)| Train Acc: 65.77%
	 Val. Loss: 0.640) |  Val. Acc: 63.16%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.631)| Train Acc: 65.68%
	 Val. Loss: 0.605) |  Val. Acc: 70.86%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.619)| Train Acc: 67.67%
	 Val. Loss: 0.607) |  Val. Acc: 69.66%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss: 0.630)| Train Acc: 66.13%
	 Val. Loss: 0.627) |  Val. Acc: 70.73%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.632)| Train Acc: 64.75%
	 Val. Loss: 0.600) |  Val. Acc: 71.26%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 0.624)| Train Acc: 65.84%
	 Val. Loss: 0.605) |  Val. Acc: 69.95%
Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.625)| Train Acc: 66.06%
	 Val. Loss: 0.612) |  Val. Acc: 69.45%
Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.626)| Train Acc: 65.82%
	 Val. Loss: 0.609) |  Val. Acc: 70.95%
Epoch: 10 | Epoch T